# L3 M2: Security_Access_Control — Authentication & Identity Management

## Learning Arc

**Duration:** 40-45 minutes  
**Prerequisites:** Generic CCC M1-M4, GCC Compliance M1.1-M1.4, FastAPI middleware concepts

### What You'll Learn

By completing this notebook, you will:

1. **Implement OAuth 2.0/OIDC authentication flow** with enterprise Identity Providers (Okta, Azure AD) using authorization code flow with PKCE
2. **Design RBAC systems** with multi-tenant awareness, four standard roles (Admin, Developer, Analyst, Viewer), and permission inheritance
3. **Configure MFA enforcement** with TOTP codes and hardware token support for privileged operations
4. **Build session management** with Redis-backed token storage, TTL matching JWT expiration, and concurrent session limits
5. **Prevent session hijacking** through IP validation, User-Agent fingerprinting, and secure session metadata
6. **Validate JWT tokens** with signature verification (check signature BEFORE claims), expiration handling, and issuer/audience validation
7. **Enforce tenant isolation** at the query level to prevent cross-tenant data leakage in multi-tenant environments

### Prerequisites

- Generic CCC M1-M4 complete
- GCC Compliance M1.1-M1.4 complete
- Python 3.10+
- FastAPI middleware concepts
- Basic understanding of OAuth 2.0 and JWT

### Real-World Context

A GCC RAG system serving 50+ business units experienced:
- **Cross-tenant data leaks** (user from tenant A accessed tenant B documents)
- **Failed SOC 2 audit** (no MFA enforcement, weak session management)
- **$3.2M contract loss** (customer withdrew due to security concerns)

This module implements the authentication layer to prevent such failures.

In [ ]:
# Environment Setup and Mode Detection
import os
import sys

# Add parent directory to path for imports
sys.path.insert(0, os.path.abspath('..'))

# Check environment mode
OFFLINE_MODE = os.getenv('OFFLINE_MODE', 'true').lower() == 'true'

print("="*60)
print("L3 M2: Security_Access_Control")
print("Authentication & Identity Management")
print("="*60)
print()

if OFFLINE_MODE:
    print("⚠️  Running in OFFLINE mode")
    print("   This module demonstrates OAuth 2.0/OIDC authentication")
    print("   OAuth requires IdP integration - simulated responses will be used")
    print()
    print("   To enable real OAuth:")
    print("   1. Register OAuth app with Okta/Azure AD/Auth0")
    print("   2. Configure credentials in .env file")
    print("   3. Set OFFLINE_MODE=false in .env")
else:
    print("✓ OAuth services configured")
    print("  External Identity Provider integration enabled")

print()
print("Note: This notebook demonstrates authentication patterns")
print("      using simulated IdP responses for learning purposes")
print("="*60)

## Section 1: Authentication vs. Authorization

**Authentication** answers "Who are you?" — verifying user identity through credentials, tokens, or biometrics.

**Authorization** answers "What can you do?" — determining permissions after identity is established.

### Critical Rule: Authenticate BEFORE Authorize

Never check permissions without validating identity first!

In [ ]:
# Import core modules
from src.l3_m2_security_access_control import (
    OAuthClient,
    JWTValidator,
    RBACEngine,
    SessionManager,
    generate_pkce_pair,
    validate_tenant_isolation,
    check_permission,
)

print("✓ Core modules imported successfully")
print("  • OAuthClient: OAuth 2.0 / OIDC integration")
print("  • JWTValidator: Token validation with signature verification")
print("  • RBACEngine: Role-Based Access Control")
print("  • SessionManager: Session management with Redis")
print("  • Utility functions: PKCE, tenant isolation, permissions")

## Section 2: OAuth 2.0 Client Setup

OAuth 2.0 is a delegated authorization framework that enables applications to access user resources without exposing passwords.

**OIDC (OpenID Connect)** extends OAuth 2.0 to add authentication, providing identity tokens alongside access tokens.

### Authorization Code Flow with PKCE

1. User clicks "Login with Okta"
2. System redirects to Identity Provider with `code_challenge`
3. User authenticates at IdP (username/password + MFA)
4. IdP redirects back with authorization `code`
5. System exchanges `code` + `code_verifier` for tokens
6. JWT token issued to user

In [ ]:
# Initialize OAuth Client
oauth_config = {
    "client_id": "demo_client_id",
    "client_secret": "demo_client_secret",
    "redirect_uri": "http://localhost:8000/auth/callback",
    "authorization_endpoint": "https://demo-idp.okta.com/oauth2/v1/authorize",
    "token_endpoint": "https://demo-idp.okta.com/oauth2/v1/token",
    "userinfo_endpoint": "https://demo-idp.okta.com/oauth2/v1/userinfo",
    "issuer": "https://demo-idp.okta.com",
}

oauth_client = OAuthClient(oauth_config)

print("✓ OAuth Client initialized")
print(f"  • Issuer: {oauth_client.issuer}")
print(f"  • Client ID: {oauth_client.client_id}")
print(f"  • Redirect URI: {oauth_client.redirect_uri}")

## Section 3: PKCE (Proof Key for Code Exchange)

PKCE prevents authorization code interception attacks by requiring the original `code_verifier` to exchange the code.

**How it works:**
1. Generate random `code_verifier` (128 chars)
2. Create `code_challenge` = BASE64URL(SHA256(code_verifier))
3. Send `code_challenge` in authorization request
4. IdP stores `code_challenge` with authorization code
5. Send `code_verifier` in token exchange request
6. IdP verifies: SHA256(code_verifier) == code_challenge

In [ ]:
# Generate PKCE Pair
code_verifier, code_challenge = generate_pkce_pair()

print("✓ PKCE pair generated")
print(f"  • code_verifier (length): {len(code_verifier)} chars")
print(f"  • code_verifier (sample): {code_verifier[:50]}...")
print(f"  • code_challenge: {code_challenge}")
print()
print("Security: code_verifier is kept secret, code_challenge is sent to IdP")

## Section 4: OAuth Authorization URL Generation

The authorization URL redirects the user to the Identity Provider for authentication.

In [ ]:
import secrets

# Generate state for CSRF protection
state = secrets.token_urlsafe(32)

# Get authorization URL
auth_url = oauth_client.get_authorize_url(state, code_challenge)

print("✓ Authorization URL generated")
print()
print("Redirect user to:")
print(auth_url[:100] + "...")
print()
print("URL Components:")
print(f"  • client_id: {oauth_config['client_id']}")
print(f"  • redirect_uri: {oauth_config['redirect_uri']}")
print(f"  • response_type: code")
print(f"  • scope: openid profile email")
print(f"  • state: {state[:20]}... (CSRF protection)")
print(f"  • code_challenge: {code_challenge}")
print(f"  • code_challenge_method: S256")

## Section 5: Token Exchange

After user authenticates at IdP, the system exchanges the authorization code for tokens.

In [ ]:
# Simulate authorization code from IdP callback
authorization_code = "simulated_auth_code_from_idp"

# Exchange code for tokens
tokens = oauth_client.exchange_code_for_tokens(authorization_code, code_verifier)

print("✓ Token exchange successful")
print()
print("Received tokens:")
print(f"  • access_token: {tokens['access_token'][:50]}...")
print(f"  • id_token: {tokens['id_token'][:50]}...")
print(f"  • refresh_token: {tokens['refresh_token'][:50]}...")
print(f"  • expires_in: {tokens['expires_in']} seconds")
print(f"  • token_type: {tokens['token_type']}")

## Section 6: User Info Retrieval

Use the access token to retrieve user profile from the OIDC userinfo endpoint.

In [ ]:
# Get user info using access token
user_info = oauth_client.get_user_info(tokens['access_token'])

print("✓ User info retrieved")
print()
print("User Profile:")
print(f"  • User ID (sub): {user_info['sub']}")
print(f"  • Email: {user_info['email']}")
print(f"  • Name: {user_info['name']}")
print(f"  • Tenant ID: {user_info['tenant_id']}")
print(f"  • Roles: {user_info['roles']}")
print(f"  • Email Verified: {user_info['email_verified']}")
print(f"  • MFA Enabled: {user_info['mfa_enabled']}")
print()
print("Note: tenant_id and roles are custom claims added to user profile")

## Section 7: Role-Based Access Control (RBAC)

RBAC assigns permissions to roles, then assigns roles to users.

### Four Standard Roles:

- **Admin:** Full system access (read, write, delete, manage_users, manage_tenants)
- **Developer:** Read, write, execute
- **Analyst:** Read, query
- **Viewer:** Read only

In [ ]:
# Initialize RBAC Engine
rbac = RBACEngine()

print("✓ RBAC Engine initialized")
print()
print("Role Definitions:")
for role, permissions in rbac.roles.items():
    print(f"  • {role}: {permissions}")

In [ ]:
# Test RBAC Permission Checks
print("Permission Checks:")
print()

# Admin has all permissions
print("Admin Role:")
print(f"  • Can read? {rbac.check_permission(['Admin'], 'read')}")
print(f"  • Can write? {rbac.check_permission(['Admin'], 'write')}")
print(f"  • Can delete? {rbac.check_permission(['Admin'], 'delete')}")
print(f"  • Can manage_users? {rbac.check_permission(['Admin'], 'manage_users')}")
print()

# Developer has read, write, execute
print("Developer Role:")
print(f"  • Can read? {rbac.check_permission(['Developer'], 'read')}")
print(f"  • Can write? {rbac.check_permission(['Developer'], 'write')}")
print(f"  • Can execute? {rbac.check_permission(['Developer'], 'execute')}")
print(f"  • Can delete? {rbac.check_permission(['Developer'], 'delete')}")
print()

# Viewer has read only
print("Viewer Role:")
print(f"  • Can read? {rbac.check_permission(['Viewer'], 'read')}")
print(f"  • Can write? {rbac.check_permission(['Viewer'], 'write')}")

## Section 8: Tenant Isolation

**CRITICAL:** Every query must validate `user.tenant_id == document.tenant_id`

This prevents cross-tenant data leakage in multi-tenant systems.

In [ ]:
# Test Tenant Isolation
print("Tenant Isolation Tests:")
print()

# Same tenant - ALLOWED
user_tenant = "tenant_abc"
document_tenant = "tenant_abc"

try:
    result = rbac.validate_tenant_isolation(user_tenant, document_tenant)
    print(f"✓ Same tenant access: ALLOWED")
    print(f"  User tenant: {user_tenant}")
    print(f"  Document tenant: {document_tenant}")
    print(f"  Result: Access granted")
except ValueError as e:
    print(f"✗ Access denied: {e}")

print()

# Different tenant - DENIED
user_tenant = "tenant_abc"
document_tenant = "tenant_xyz"

try:
    result = rbac.validate_tenant_isolation(user_tenant, document_tenant)
    print(f"✗ Cross-tenant access: ALLOWED (SECURITY BUG!)")
except ValueError as e:
    print(f"✓ Cross-tenant access: DENIED")
    print(f"  User tenant: {user_tenant}")
    print(f"  Document tenant: {document_tenant}")
    print(f"  Result: {e}")

## Section 9: Session Management

Session management includes:
- Redis-backed storage with TTL
- Security fingerprints (IP address, User-Agent)
- Session hijacking detection
- Concurrent session limits

In [ ]:
# Initialize Session Manager
session_config = {
    "redis_host": "localhost",
    "redis_port": 6379,
    "session_timeout": 3600,
    "max_concurrent_sessions": 3,
}

session_mgr = SessionManager(session_config)

print("✓ Session Manager initialized")
print(f"  • Redis: {session_mgr.redis_host}:{session_mgr.redis_port}")
print(f"  • Session timeout: {session_mgr.session_timeout} seconds")
print(f"  • Max concurrent sessions: {session_mgr.max_concurrent_sessions}")

In [ ]:
# Create Session
session_id = session_mgr.create_session(
    user_id=user_info['sub'],
    token=tokens['access_token'],
    ip_address="192.168.1.100",
    user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
)

print("✓ Session created")
print(f"  • Session ID: {session_id[:20]}...")
print(f"  • User ID: {user_info['sub']}")
print(f"  • IP Address: 192.168.1.100")
print(f"  • User-Agent: Mozilla/5.0...")
print(f"  • Expires in: {session_config['session_timeout']} seconds")

In [ ]:
# Validate Session - Same IP (VALID)
print("Session Validation Test 1: Same IP")
try:
    is_valid = session_mgr.validate_session(
        session_id=session_id,
        ip_address="192.168.1.100",
        user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    )
    print(f"  • Result: {'VALID' if is_valid else 'INVALID'}")
    print(f"  • IP match: ✓")
    print(f"  • User-Agent match: ✓")
except ValueError as e:
    print(f"  • Result: INVALID")
    print(f"  • Error: {e}")

print()

# Validate Session - Different IP (HIJACKING DETECTED)
print("Session Validation Test 2: Different IP")
try:
    is_valid = session_mgr.validate_session(
        session_id=session_id,
        ip_address="10.0.0.50",  # Different IP!
        user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    )
    print(f"  • Result: {'VALID' if is_valid else 'INVALID'}")
except ValueError as e:
    print(f"  • Result: INVALID - Session hijacking detected")
    print(f"  • IP mismatch: 192.168.1.100 → 10.0.0.50")
    print(f"  • Error: {e}")

## Section 10: Session Hijacking Detection

The system detects hijacking through:
1. **IP Address Validation:** Reject if IP changes (strict mode)
2. **User-Agent Validation:** Log warning if User-Agent changes (can change legitimately)

In [ ]:
from src.l3_m2_security_access_control import detect_session_hijacking

# Original session metadata
session_data = {
    "ip_address": "192.168.1.100",
    "user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
}

print("Session Hijacking Detection Tests:")
print()

# Test 1: No change (valid)
is_hijacked = detect_session_hijacking(
    session_data,
    current_ip="192.168.1.100",
    current_user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
)
print(f"Test 1 - No changes: {'HIJACKED' if is_hijacked else 'VALID'}")

# Test 2: IP changed (hijacking!)
is_hijacked = detect_session_hijacking(
    session_data,
    current_ip="10.0.0.50",
    current_user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
)
print(f"Test 2 - IP changed: {'HIJACKED ⚠️' if is_hijacked else 'VALID'}")

# Test 3: User-Agent changed (low risk)
is_hijacked = detect_session_hijacking(
    session_data,
    current_ip="192.168.1.100",
    current_user_agent="Chrome/120.0"
)
print(f"Test 3 - User-Agent changed: {'HIJACKED' if is_hijacked else 'VALID (low-risk warning logged)'}")

## Section 11: Complete Request Validation Workflow

Putting it all together: Validate a complete request with:
1. JWT token validation
2. RBAC permission check
3. Tenant isolation enforcement
4. Session validation

In [ ]:
def validate_request(
    user_roles,
    user_tenant_id,
    required_permission,
    resource_tenant_id,
    session_id,
    current_ip,
    current_user_agent
):
    """
    Complete request validation workflow.
    
    Returns:
        (bool, str): (is_authorized, message)
    """
    # Step 1: Check RBAC permissions
    if not check_permission(user_roles, required_permission):
        return False, f"Permission denied - {user_roles} lacks '{required_permission}'"
    
    # Step 2: Validate tenant isolation
    try:
        validate_tenant_isolation(user_tenant_id, resource_tenant_id)
    except ValueError as e:
        return False, f"Tenant isolation violation - {e}"
    
    # Step 3: Validate session (simulated)
    # In production: validate_session(session_id, current_ip, current_user_agent)
    
    return True, "Request authorized"


# Test Case 1: Valid request
print("Test Case 1: Valid Request")
is_authorized, message = validate_request(
    user_roles=["Developer"],
    user_tenant_id="tenant_abc",
    required_permission="write",
    resource_tenant_id="tenant_abc",
    session_id=session_id,
    current_ip="192.168.1.100",
    current_user_agent="Mozilla/5.0"
)
print(f"  • Result: {'✓ AUTHORIZED' if is_authorized else '✗ DENIED'}")
print(f"  • Message: {message}")
print()

# Test Case 2: Insufficient permissions
print("Test Case 2: Insufficient Permissions")
is_authorized, message = validate_request(
    user_roles=["Viewer"],  # Viewer can't write
    user_tenant_id="tenant_abc",
    required_permission="write",
    resource_tenant_id="tenant_abc",
    session_id=session_id,
    current_ip="192.168.1.100",
    current_user_agent="Mozilla/5.0"
)
print(f"  • Result: {'✓ AUTHORIZED' if is_authorized else '✗ DENIED'}")
print(f"  • Message: {message}")
print()

# Test Case 3: Cross-tenant access
print("Test Case 3: Cross-Tenant Access")
is_authorized, message = validate_request(
    user_roles=["Developer"],
    user_tenant_id="tenant_abc",
    required_permission="write",
    resource_tenant_id="tenant_xyz",  # Different tenant!
    session_id=session_id,
    current_ip="192.168.1.100",
    current_user_agent="Mozilla/5.0"
)
print(f"  • Result: {'✓ AUTHORIZED' if is_authorized else '✗ DENIED'}")
print(f"  • Message: {message}")

## Section 12: Summary & Key Takeaways

### What We Learned

1. **OAuth 2.0/OIDC** enables enterprise SSO without password storage
2. **PKCE** prevents authorization code interception attacks
3. **JWT tokens** provide cryptographically signed identity assertions
4. **RBAC** scales permission management through role assignments
5. **Tenant isolation** prevents cross-tenant data leakage (critical!)
6. **Session management** detects hijacking through IP/User-Agent validation
7. **Signature verification BEFORE claims** prevents token tampering

### Critical Rules

- ✅ **Always** verify JWT signature BEFORE checking claims
- ✅ **Always** validate `user.tenant_id == document.tenant_id` in queries
- ✅ **Always** authenticate BEFORE authorizing
- ✅ **Never** decode tokens without signature verification in production
- ✅ **Never** skip tenant isolation checks

### Production Checklist

- [ ] OAuth app registered with Identity Provider
- [ ] Client credentials configured in .env
- [ ] Redis running for session storage
- [ ] PostgreSQL configured for user/tenant metadata
- [ ] JWT secret keys generated securely
- [ ] HTTPS enabled (TLS certificates)
- [ ] Audit logging configured
- [ ] MFA enforcement for Admin role
- [ ] Session timeout configured (default: 1 hour)
- [ ] Concurrent session limits enforced

### Next Steps

- **M2.2:** Authorization patterns and advanced RBAC
- **M2.3:** Multi-Factor Authentication implementation
- **M2.4:** Audit logging and compliance reporting

In [ ]:
# Final Summary
print("="*60)
print("L3 M2: Security_Access_Control - Complete!")
print("="*60)
print()
print("✓ OAuth 2.0/OIDC authentication flow")
print("✓ PKCE code challenge/verifier generation")
print("✓ Role-Based Access Control (4 roles)")
print("✓ Tenant isolation enforcement")
print("✓ Session management with hijacking detection")
print("✓ Complete request validation workflow")
print()
print("For production deployment:")
print("  • Configure OAuth credentials in .env")
print("  • Start Redis: docker run -p 6379:6379 redis:7-alpine")
print("  • Run API: uvicorn app:app --reload")
print("  • Visit: http://localhost:8000/docs")
print()
print("="*60)